In [28]:
#this jupiter notebook rassemble my work in order
#first we import every libraries that we need
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
#then we create the file were the datas are gonna go
test = f"./fulldatas.json"
#we initiate the driver for the selenium part
def driver_gen():
    driver = webdriver.Chrome(executable_path="driver/chromedriver")
    return driver

In [29]:
# we get data from the list of links func 
def clean_data(listed):
    house_number = 1
    dict_of_informations= {}
    list_of_link = listed
    for lk in list_of_link : 
        r = requests.get(lk)
        content = r.content
        property_details_page = BeautifulSoup(content,"html.parser")
        dict_of_data= {}
        #we clean the datas by finding tags
        for tr in property_details_page.find_all('tr', attrs={'class':'classified-table__row'}):
            #these are the tage were our infos are
            value = tr.find('th')
            data = tr.find('td')
            if value != None :
                #we clean them using regex yay !
                title = re.sub(r'\s*<[^>]*>\s*','',str(value))
                if data != None : 
                    datas = re.sub(r'\s*<[^>]*>\s*','',str(data))
                    # we delete the big space to replace them with 1 only
                    datass = " ".join(datas.split())
                    #we store data in the dict
                    dict_of_data[title] = datass
                else: 
                    #if there are no datas we write none
                    dict_of_data[title] = None
            else: 
                continue
        #we put the datas in a another dict with all the house listed
        dict_of_informations[house_number] = dict_of_data
        house_number += 1 
    return dict_of_informations
    #right now the data is are dictionaries in a dictionary

In [30]:
# we list all the links to scrape them after
def get_links_and_clean(iterations, driver):
    list_of_link = []
    #still doing by page number is gonna change
    for num in iterations:
        page_num = str(num) + "&orderBy=relevance"
        url = (
                "https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=" + page_num
                )
        driver.get(url)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        dict_of_informations = {}
        for link in soup.find_all("a", class_="card__title-link") : 
            list_of_link.append(link['href'])
    driver.quit()
    #calling clean function to clean and return clean data
    #not the good way i think incorporate links finding in data cleaning
    cleaned = clean_data(list_of_link)
    return cleaned


In [31]:
#saving the data in a json file
def saving_datas(datas):
    with open(test, 'a') as file:
        json.dump(str(datas), file, indent=4)

In [ ]:
#creating of the thread pool
#need to return on this function to divide the selenium and requests tasks
# map 300 links to the 36*300 houses to the 8 or 16 drivers and threads
def pool():
    # 1 driver by thread (we have four thread here)
    drivers = [driver_gen() for _ in range(8)]
    #division of the number of pages by the number of threads 
    division = np.array_split(np.arange(1,16),8)
    #creation of a pool of threads
    with ThreadPoolExecutor(max_workers=8) as executor :
        #it is where it needs refinement
        results = executor.map(get_links_and_clean,division,drivers) 
        for result in results:
            saving_datas(result)
            print(result)

pool()


#bug = i get the same house over and over in my final dictionary 
#but all the links are different ... Why ? 